<a href="https://colab.research.google.com/github/allythe/NeuralStyleTransfer_VGG16_VGG19/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import resources
import os
from PIL import Image  # Import the Image class from the PIL library for image processing
from io import BytesIO  # Import BytesIO for handling input/output streams
import matplotlib.pyplot as plt  # Import pyplot for plotting
import numpy as np  # Import NumPy for numerical computing
import skimage
import math
from skimage.metrics import structural_similarity as ssim

from tqdm import tqdm
import torch  # Import PyTorch
import torch.optim as optim  # Import the optimization module from PyTorch
import requests  # Import the requests library for making HTTP requests
from torchvision import transforms, models  # Import transforms and models from torchvision for computer vision
from typing import Optional, Tuple  # Import type hints
import logging
from torchvision.models import VGG16_Weights, VGG19_Weights

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
!ls "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/implementation/"

Mounted at /content/drive/
VGG16.ipynb  VGG19.ipynb


## Test data

In [ ]:
content_dir = "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark"
style_dir   = "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style"

def get_first_n_images(folder, n=10):
    images = []
    for entry in os.scandir(folder):
        if entry.is_file() and entry.name.lower().endswith((".png", ".jpg", ".jpeg")):
            images.append(entry.path)

    return images[-10:]

content_images = get_first_n_images(content_dir)
style_images   = get_first_n_images(style_dir)

pairs = list(zip(content_images, style_images))

print("Generated 10 pairs:")
for i, (c, s) in enumerate(pairs):
    print(f"{i+1}. CONTENT = {c}   |   STYLE = {s}")

Generated 10 pairs:
1. CONTENT = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/desert1024.jpg   |   STYLE = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/TheKiss_Klimt.png
2. CONTENT = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/mac1024.jpg   |   STYLE = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/StarryNight_VanGogh.png
3. CONTENT = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/cat1024.jpg   |   STYLE = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/ThreeFishingBoats_Monet.png
4. CONTENT = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/athletes1024.jpg   |   STYLE = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/RitmoPlastico_Severini.png
5. CONTENT = /content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/tomato1024.jpg 

## NST implementation

In [ ]:
def calculate_ssim_score(content_img_np, generated_img_np):
    score = ssim(content_img_np, generated_img_np,
                 channel_axis=-1,
                 data_range=1.0,
                 win_size=7)
    return score



In [ ]:
class NeuralStyleTransfer:
    def __init__(self, vgg_type, content_layers, style_layers, pooling = "max", lr = 0.003):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.pooling = pooling

        self.logger = self._init_logger()
        self.vgg_type = vgg_type
        self.vgg = self._init_vgg()
        self.vgg.to(self.device)
        self.lr = lr
        self.content_layers = content_layers
        self.style_layers = style_layers
        self.style_layer_weights = {'conv1_1': 1.0, 'conv2_1': 0.75, 'conv3_1': 0.2,
                         'conv4_1': 0.2, 'conv5_1': 0.2}
        self.content_layer_weights = {'conv1_2': 1.0, 'conv2_2': 1, 'conv3_2': 1,
                         'conv4_2': 1, 'conv5_2': 1}

        self.logger.info(f"VGG model {self.vgg_type} initialized")

    def _init_vgg(self):
        if self.vgg_type == "vgg16":
            vgg = models.vgg16(weights=VGG16_Weights.DEFAULT).features
        elif self.vgg_type == "vgg19":
            vgg = models.vgg19(weights=VGG19_Weights.DEFAULT).features
        else:
          raise ValueError(f"Unknown VGG type: {self.vgg_type}")

        if self.pooling == "avg":
            for i, layer in enumerate(vgg):
                if isinstance(layer, torch.nn.MaxPool2d):
                    vgg[i] = torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
            self.logger.info("Replaced MaxPool with AvgPool")



        for param in vgg.parameters():
            param.requires_grad_(False)



        return vgg

    def _init_logger(self):
        logger = logging.getLogger(__name__)
        logging.basicConfig(format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                      datefmt='%H:%M:%S',
                      level=logging.INFO,
                      force=True )
        logger.info("Logger created")
        return logger

    def print_vgg_layers(self):
        print(self.vgg)

    def _check_image(self, img_tensor, name="image"):
        self.logger.info(f"Checking {name}...")

        if not isinstance(img_tensor, torch.Tensor):
            raise TypeError(f"{name} is not a tensor")


        # Check expected shape (1, 3, 256, 256)
        if img_tensor.shape[1:] != (3, 256, 256):
            self.logger.warning(f"{name} has unexpected shape: {img_tensor.shape}")
        else:
            self.logger.info(f"{name} shape is correct.")

    def _compute_gram_matrix(self, input):
        b, d, h, w = input.size()
        features = input.view(b * d, h * w)
        gram = torch.mm(features, features.t())
        return gram
        # Convert PyTorch tensors (1, 3, H, W) → NumPy (H, W, 3)
    def tensor_to_numpy(self, tensor):
        img = tensor.detach().cpu().clone()
        img = img.squeeze(0).permute(1, 2, 0).numpy()
        img = img.clip(0, 1)
        return img

    def _extract_features(self, image) -> dict:
        needed_layers = set(self.content_layers.values()) | set(self.style_layers.values())

        if self.vgg_type == "vgg16":
            layer_map = {
                '0': 'conv1_1', '1': 'relu1_1',
                '2': 'conv1_2', '3': 'relu1_2',
                '5': 'conv2_1', '6': 'relu2_1',
                '7': 'conv2_2', '8': 'relu2_2',
                '10': 'conv3_1', '11': 'relu3_1',
                '12': 'conv3_2', '13': 'relu3_2',
                '14': 'conv3_3', '15': 'relu3_3',
                '17': 'conv4_1', '18': 'relu4_1',
                '19': 'conv4_2', '20': 'relu4_2',
                '21': 'conv4_3', '22': 'relu4_3',
                '24': 'conv5_1', '25': 'relu5_1',
                '26': 'conv5_2', '27': 'relu5_2',
                '28': 'conv5_3', '29': 'relu5_3',
            }

        features = {}
        x = image

        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in layer_map:
                lname = layer_map[name]
                if lname in needed_layers:
                    features[lname] = x

        return features

    def _im_convert(self, tensor):
        image = tensor.cpu().clone().detach().squeeze(0)  # remove batch dimension
        image = image.numpy().transpose(1, 2, 0)  # C x H x W -> H x W x C
        # Denormalize
        image = image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
        image = image.clip(0, 1)
        return image


    def _load_img(self, img_path):
        img = skimage.io.imread(img_path)
        img = Image.fromarray(img)

        in_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

        img = in_transform(img)[:3, :, :].unsqueeze(0).to(self.device)
        return img


    def __call__(self, context_img_path, style_img_path, epochs = 2000, content_weight = 100, style_weight=1, show_every = 100):
        content_img = self._load_img(context_img_path)
        style_img = self._load_img(style_img_path)

        self._check_image(content_img, "content image")
        self._check_image(style_img, "style image")

        content_features = self._extract_features(content_img)
        style_features = self._extract_features(style_img)
        style_grams = {layer: self._compute_gram_matrix(style_features[layer]) for layer in  self.style_layers.values()}

        target = content_img.clone() * 0.3 + torch.randn_like(content_img) * 0.5

        target = target.to(self.device)
        target.requires_grad_(True)

        optimizer = optim.Adam([target], lr=self.lr)

        total_losses = []
        content_losses = []
        style_losses = []


        tqdm_bar = tqdm(range(1, epochs + 1), desc="NST Progress", ncols=200)
        for ii in tqdm_bar:
            target_features = self._extract_features(target)

            # content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)
            content_loss = 0

            for layer in self.content_layers.values():

                target_feature = target_features[layer]
                content_feature = content_features[layer]

                w = self.content_layer_weights[layer]

                content_loss += w * torch.mean((target_feature - content_feature)**2)

            # Style loss
            style_loss = 0

            for layer in self.style_layers.values():

                target_feature = target_features[layer]
                target_gram = self._compute_gram_matrix(target_feature)
                _, d, h, w = target_feature.shape
                style_gram = style_grams[layer]
                layer_loss = self.style_layer_weights[layer] * torch.mean((target_gram - style_gram)**2)
                style_loss += layer_loss / (d * h * w)


            total_loss = content_weight * content_loss + style_weight * style_loss

            content_losses.append(content_loss.item())
            style_losses.append(style_loss.item())
            total_losses.append(total_loss.item())

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            tqdm_bar.set_postfix({
                  "Epoch": f"{ii}",
                  "Total": f"{total_loss.item():.2f}",
                  "Content": f"{(content_loss.item()*content_weight):.2f}",
                  "Style": f"{(style_loss.item()*style_weight):.2f}"
              })

            # Plot intermediate result every show_every steps
            if ii % show_every == 0:
                plt.figure(figsize=(15,5))

                # Content
                plt.subplot(1, 3, 1)
                plt.imshow(self._im_convert(content_img))
                plt.title("Content")
                plt.axis('off')

                # Style
                plt.subplot(1, 3, 2)
                plt.imshow(self._im_convert(style_img))
                plt.title("Style")
                plt.axis('off')

                # Current Target
                plt.subplot(1, 3, 3)
                plt.imshow(self._im_convert(target))
                plt.title(f"Target Step {ii}")
                plt.axis('off')

                plt.show()
        ssim = calculate_ssim_score(
        self.tensor_to_numpy(content_img),
        self.tensor_to_numpy(target)
        )
        ssim = float(ssim)



        return (
                self._im_convert(target),
                {
                    "content_loss": content_losses[-1],
                    "style_loss": style_losses[-1],
                    "total_loss": total_losses[-1],
                    "ssim_loss": ssim,
                }
               )

## Change style_weight and content_weight

In [ ]:
layers = {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '17': 'conv4_1', '19': 'conv4_2', '24': 'conv5_1'}

nst = NeuralStyleTransfer(vgg_type="vgg16", layers = layers)

weight_configs = [
    (1, 1),
    (100, 1),
    (10000, 1),
    (1000000, 1),
    (1, 100),
    (1, 10000),
    (1, 1000000)
]

output_dir = "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/results"
os.makedirs(output_dir, exist_ok=True)

for pair_index, (content_img_path, style_img_path) in enumerate(pairs):

    print(f"\n====================")
    print(f"PAIR {pair_index+1}/{len(pairs)}")
    print(f"Content: {content_img_path}")
    print(f"Style:   {style_img_path}")
    print(f"====================\n")

    pair_dir = os.path.join(output_dir, f"pair_{pair_index+1}")
    os.makedirs(pair_dir, exist_ok=True)

    pair_results = []

    for cw, sw in weight_configs:
        fname = f"cw{cw}_sw{sw}.png"
        save_path = os.path.join(pair_dir, fname)

        if os.path.exists(save_path):
            print(f"Skipping existing file: {save_path}")
            # load existing image for grid
            img = skimage.io.imread(save_path)
            pair_results.append((cw, sw, img))
            continue

        print(f"--- Running NST: content={cw}, style={sw} ---")
        result_img = nst(
            context_img_path=content_img_path,
            style_img_path=style_img_path,
            epochs=1500,
            content_weight=cw,
            style_weight=sw,
            show_every=999999
        )

        plt.imsave(save_path, result_img)
        print(f"Saved: {save_path}")
        pair_results.append((cw, sw, result_img))

    # ====== Create grid ======
    fig = plt.figure(figsize=(12, 12))

    # 1st row: content, style, first result
    ax = fig.add_subplot(3, 3, 1)
    ax.imshow(skimage.io.imread(content_img_path))
    ax.set_title("Content")
    ax.axis("off")

    ax = fig.add_subplot(3, 3, 2)
    ax.imshow(skimage.io.imread(style_img_path))
    ax.set_title("Style")
    ax.axis("off")

    cw, sw, img = pair_results[0]
    ax = fig.add_subplot(3, 3, 3)
    ax.imshow(img)
    ax.set_title(f"cw={cw}, sw={sw}")
    ax.axis("off")

    # 2nd row: results 2,3,4
    for i, idx in enumerate([1, 2, 3]):
        cw, sw, img = pair_results[idx]
        ax = fig.add_subplot(3, 3, 4+i)
        ax.imshow(img)
        ax.set_title(f"cw={cw}, sw={sw}")
        ax.axis("off")

    # 3rd row: results 5,6,7
    for i, idx in enumerate([4, 5, 6]):
        cw, sw, img = pair_results[idx]
        ax = fig.add_subplot(3, 3, 7+i)
        ax.imshow(img)
        ax.set_title(f"cw={cw}, sw={sw}")
        ax.axis("off")

    plt.tight_layout()
    grid_path = os.path.join(pair_dir, "grid.png")
    plt.savefig(grid_path, dpi=200)
    plt.close()
    print(f"Saved GRID: {grid_path}")

print("\n🎉 ALL DONE! Grids saved to:", output_dir)

TypeError: NeuralStyleTransfer.__init__() got an unexpected keyword argument 'layers'

# Change content and style layers

In [ ]:
style_layers_list = [
    {'0': 'conv1_1'},
    {'0': 'conv1_1', '5': 'conv2_1' },
    {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1'},
    {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '17': 'conv4_1'},
    {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '17': 'conv4_1', '24': 'conv5_1'}
]

content_layers_list = [
    {'2': 'conv1_2'},

    {'2': 'conv1_2', '7': 'conv2_2'},
    {'2': 'conv1_2', '7': 'conv2_2', '12': 'conv3_2'},
    {'2': 'conv1_2', '7': 'conv2_2', '12': 'conv3_2', '19': 'conv4_2'},
]

output_dir = "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers"
os.makedirs(output_dir, exist_ok=True)

for pair_index, (content_img_path, style_img_path) in enumerate(pairs):

    print(f"\n=== Processing PAIR {pair_index+1}/{len(pairs)} ===")
    print(content_img_path)
    print(style_img_path)

    pair_dir = os.path.join(output_dir, f"pair_{pair_index+1}")
    os.makedirs(pair_dir, exist_ok=True)

    results = []  # store (img, loss_dict, content_layer_index, style_layer_index)

    for c_l_index, content_layers in enumerate(content_layers_list):
        for s_l_index, style_layers in enumerate(style_layers_list):

            print(f"\nRunning NST with content_layers {c_l_index+1}/{len(content_layers_list)}, "
                  f"style_layers {s_l_index+1}/{len(style_layers_list)}")
            print("Content layers:", content_layers)
            print("Style layers:", style_layers)

            save_path = os.path.join(pair_dir, f"layers_C{c_l_index+1}_S{s_l_index+1}.png")

            nst = NeuralStyleTransfer(
                vgg_type="vgg16",
                content_layers=content_layers,
                style_layers=style_layers
            )

            if os.path.exists(save_path):
                print(f"Loading existing file: {save_path}")
                img = skimage.io.imread(save_path)
                loss_dict = None
            else:
                img, loss_dict = nst(
                    context_img_path=content_img_path,
                    style_img_path=style_img_path,
                    epochs=1500,
                    content_weight=1,
                    style_weight=1,
                    show_every=999999
                )
                if img is not None:
                    plt.imsave(save_path, img)
                    print(f"Saved: {save_path}")

            results.append((img, loss_dict, c_l_index+1, s_l_index+1))

    # Plot grid
    n_results = len(results)
    cols = 5
    rows = math.ceil((n_results + 2) / cols)  # +2 for content & style images
    fig = plt.figure(figsize=(4*cols, 4*rows))

    # Content image
    ax = fig.add_subplot(rows, cols, 1)
    ax.imshow(skimage.io.imread(content_img_path))
    ax.set_title("Content")
    ax.axis("off")

    # Style image
    ax = fig.add_subplot(rows, cols, 2)
    ax.imshow(skimage.io.imread(style_img_path))
    ax.set_title("Style")
    ax.axis("off")

    # NST results
    for idx, (img, loss_dict, c_idx, s_idx) in enumerate(results):
        ax = fig.add_subplot(rows, cols, 3 + idx)
        ax.imshow(img)
        if loss_dict is None:
            title = f"C{c_idx} S{s_idx}"
        else:
            title = f"C{c_idx} S{s_idx}\nC:{loss_dict['content_loss']:.2f}  S:{loss_dict['style_loss']:.2f}"
        ax.set_title(title)
        ax.axis("off")

    plt.tight_layout()
    grid_path = os.path.join(pair_dir, "grid.png")
    plt.savefig(grid_path, dpi=200)
    plt.close()
    print(f"Saved GRID for pair {pair_index+1}: {grid_path}")

10:38:52,462 __main__ INFO Logger created



=== Processing PAIR 1/10 ===
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/desert1024.jpg
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/TheKiss_Klimt.png

Running NST with content_layers 1/4, style_layers 1/5
Content layers: {'2': 'conv1_2'}
Style layers: {'0': 'conv1_1'}


10:38:53,954 __main__ INFO VGG model vgg16 initialized
10:38:53,995 __main__ INFO Checking content image...
10:38:53,995 __main__ INFO content image shape is correct.
10:38:53,997 __main__ INFO Checking style image...
10:38:53,998 __main__ INFO style image shape is correct.
NST Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:18<00:00, 80.97it/s, Epoch=1500, Total=0.62, Content=0.50, Style=0.12]
10:39:12,579 __main__ INFO Logger created


Saved: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers/pair_1/layers_C1_S1.png

Running NST with content_layers 1/4, style_layers 2/5
Content layers: {'2': 'conv1_2'}
Style layers: {'0': 'conv1_1', '5': 'conv2_1'}


10:39:14,745 __main__ INFO VGG model vgg16 initialized
10:39:14,800 __main__ INFO Checking content image...
10:39:14,801 __main__ INFO content image shape is correct.
10:39:14,802 __main__ INFO Checking style image...
10:39:14,802 __main__ INFO style image shape is correct.
NST Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:21<00:00, 70.35it/s, Epoch=1500, Total=3.38, Content=1.57, Style=1.81]
10:39:36,168 __main__ INFO Logger created


...

Saved: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers/pair_10/layers_C4_S3.png

Running NST with content_layers 4/4, style_layers 4/5
Content layers: {'2': 'conv1_2', '7': 'conv2_2', '12': 'conv3_2', '19': 'conv4_2'}
Style layers: {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '17': 'conv4_1'}


12:19:48,163 __main__ INFO VGG model vgg16 initialized
12:19:48,208 __main__ INFO Checking content image...
12:19:48,208 __main__ INFO content image shape is correct.
12:19:48,209 __main__ INFO Checking style image...
12:19:48,211 __main__ INFO style image shape is correct.
NST Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:32<00:00, 46.41it/s, Epoch=1500, Total=134.37, Content=76.94, Style=57.43]
12:20:20,609 __main__ INFO Logger created


Saved: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers/pair_10/layers_C4_S4.png

Running NST with content_layers 4/4, style_layers 5/5
Content layers: {'2': 'conv1_2', '7': 'conv2_2', '12': 'conv3_2', '19': 'conv4_2'}
Style layers: {'0': 'conv1_1', '5': 'conv2_1', '10': 'conv3_1', '17': 'conv4_1', '24': 'conv5_1'}


12:20:22,774 __main__ INFO VGG model vgg16 initialized
12:20:22,818 __main__ INFO Checking content image...
12:20:22,819 __main__ INFO content image shape is correct.
12:20:22,819 __main__ INFO Checking style image...
12:20:22,820 __main__ INFO style image shape is correct.
NST Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:34<00:00, 43.14it/s, Epoch=1500, Total=133.81, Content=76.47, Style=57.34]


Saved: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers/pair_10/layers_C4_S5.png
Saved GRID for pair 10: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/style_layers/pair_10/grid.png


# Change pooling - max / avg

In [ ]:
# FIXED layers
CONTENT_LAYERS = {'2': 'conv1_2'}

STYLE_LAYERS = {
    '0': 'conv1_1',
    '5': 'conv2_1',
    '10': 'conv3_1',
    '17': 'conv4_1'
}

OUTPUT_DIR = "/content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg"
os.makedirs(OUTPUT_DIR, exist_ok=True)

POOLINGS = ["max", "avg"]

# Create NST objects ONCE
nst_models = {
    pooling: NeuralStyleTransfer(
        vgg_type="vgg16",
        content_layers=CONTENT_LAYERS.copy(),
        style_layers=STYLE_LAYERS.copy(),
        pooling=pooling,
        lr=0.003
    )
    for pooling in POOLINGS
}

print("\nNeuralStyleTransfer instances initialized.\n")


for pair_idx, (content_img_path, style_img_path) in enumerate(pairs):

    print(f"\n===== PAIR {pair_idx+1}/{len(pairs)} =====")
    print(content_img_path)
    print(style_img_path)

    pair_dir = os.path.join(OUTPUT_DIR, f"pair_{pair_idx+1}")
    os.makedirs(pair_dir, exist_ok=True)

    results = []

    for pooling, nst in nst_models.items():

        print(f"\nRunning NST with {pooling.upper()} pooling...")

        save_path = os.path.join(pair_dir, f"pool_{pooling}.png")

        if os.path.exists(save_path):
            print(f"Loading existing file: {save_path}")
            img = skimage.io.imread(save_path)
            loss_dict = None
        else:
            img, loss_dict = nst(
                context_img_path=content_img_path,
                style_img_path=style_img_path,
                epochs=1500,
                content_weight=1,
                style_weight=1,
                show_every=999999
            )
            plt.imsave(save_path, img)


        results.append((img, pooling, loss_dict))

    # plot comparison

    # plot comparison (Content | Style | MAX | AVG)
    fig = plt.figure(figsize=(14, 6))

    # --- Content ---
    ax = fig.add_subplot(1, 4, 1)
    ax.imshow(skimage.io.imread(content_img_path))
    ax.set_title("Content")
    ax.axis("off")

    # --- Style ---
    ax = fig.add_subplot(1, 4, 2)
    ax.imshow(skimage.io.imread(style_img_path))
    ax.set_title("Style")
    ax.axis("off")

    # --- MAX and AVG results ---
    for i, (img, pool, loss_dict) in enumerate(results):
        ax = fig.add_subplot(1, 4, i + 3)
        ax.imshow(img)

        if loss_dict is None:
            title = f"{pool.upper()} pooling"
        else:
            title = (
                f"{pool.upper()} pooling\n"
                f"C:{loss_dict['content_loss']:.2f}  "
                f"S:{loss_dict['style_loss']:.2f}\n"
                f"SSIM:{float(loss_dict['ssim_loss']):.2f}"
            )

        ax.set_title(title)
        ax.axis("off")

    plt.tight_layout()
    grid_path = os.path.join(pair_dir, "comparison_max_vs_avg.png")
    plt.savefig(grid_path, dpi=200)
    plt.close()

    print(f"Saved pooling comparison GRID: {grid_path}")

11:17:52,410 __main__ INFO Logger created
11:17:55,360 __main__ INFO Replaced MaxPool with AvgPool
11:17:55,381 __main__ INFO VGG model vgg16 initialized
11:17:55,382 __main__ INFO Logger created
11:17:56,732 __main__ INFO Replaced MaxPool with AvgPool
11:17:56,752 __main__ INFO VGG model vgg16 initialized



NeuralStyleTransfer instances initialized.


===== PAIR 1/10 =====
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/desert1024.jpg
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/TheKiss_Klimt.png

Running NST with MAX pooling...
Loading existing file: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg/pair_1/pool_max.png

Running NST with AVG pooling...
Loading existing file: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg/pair_1/pool_avg.png


11:17:58,725 __main__ INFO Checking content image...
11:17:58,726 __main__ INFO content image shape is correct.
11:17:58,727 __main__ INFO Checking style image...
11:17:58,728 __main__ INFO style image shape is correct.


Saved pooling comparison GRID: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg/pair_1/comparison_max_vs_avg.png

===== PAIR 2/10 =====
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/mac1024.jpg
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/StarryNight_VanGogh.png

Running NST with MAX pooling...


NST Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:40<00:00, 49.98it/s, Epoch=2000, Total=3.27, Content=1.91, Style=1.36]
11:18:38,887 __main__ INFO Checking content image...
11:18:38,888 __main__ INFO content image shape is correct.
11:18:38,889 __main__ INFO Checking style image...
11:18:38,890 __main__ INFO style image shape is correct.


...

Saved pooling comparison GRID: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg/pair_9/comparison_max_vs_avg.png

===== PAIR 10/10 =====
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/content/NPRgeneral_benchmark/city1024.jpg
/content/drive/MyDrive/MIUN/DT088A_MachineLearning/images/style/IndianOrnament.png

Running NST with MAX pooling...


11:28:58,619 __main__ INFO Checking content image...
11:28:58,619 __main__ INFO content image shape is correct.
11:28:58,620 __main__ INFO Checking style image...
11:28:58,621 __main__ INFO style image shape is correct.
NST Progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:39<00:00, 50.45it/s, Epoch=2000, Total=43.97, Content=4.51, Style=39.46]
11:29:38,355 __main__ INFO Checking content image...
11:29:38,356 __main__ INFO content image shape is correct.
11:29:38,357 __main__ INFO Checking style image...
11:29:38,357 __main__ INFO style image shape is correct.



Running NST with AVG pooling...


NST Progress: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:39<00:00, 50.41it/s, Epoch=2000, Total=15.76, Content=3.81, Style=11.95]


Saved pooling comparison GRID: /content/drive/MyDrive/MIUN/DT088A_MachineLearning/results_vgg16/pooling_max_avg/pair_10/comparison_max_vs_avg.png
